# Get started with Model Registry

This notebook demonstrates how to use Evidently to:
* Generate a model performance report and calculate associated metrics.
* Log model metrics to MLFlow.
* Store the model in MLFlow as an artifact.
* Store the model performance report in MLFlow as an artifact.

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [2]:
# Download original dataset with: python src/load_data.py 

raw_data = pd.read_csv(f"../data/raw_data.csv")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [3]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [4]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

print(sample_data.shape)

(594, 17)


In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

(415, 10)
(179, 10)


## Train a  Linear Regression Model

In [6]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train) 

model_lr_path = Path('../models/model_lr.joblib')
joblib.dump(model_lr, model_lr_path)

['../models/model_lr.joblib']

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds_lr = model_lr.predict(X_test)

me = mean_squared_error(y_test, preds_lr)
mae = mean_absolute_error(y_test, preds_lr)

print(me, mae)

1951.469020528523 32.808413735394325


## Train a RandomForestRegressor Model

In [8]:
model_rf = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model_rf.fit(X_train, y_train) 

model_path = Path('../models/model_rf.joblib')
joblib.dump(model_rf, model_path)

['../models/model_rf.joblib']

In [9]:

preds_rf = model_rf.predict(X_test)

me = mean_squared_error(y_test, preds_rf)
mae = mean_absolute_error(y_test, preds_rf)

print(me, mae)

360.01809385474866 12.35653631284916


# Model Registry

## Set up MLFlow

In [10]:
# Set up MLFlow 
MLFLOW_TRACKING_URI = "http://localhost:5001"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Set up MLFlow Client
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment('4-Model-Registry')


2024/10/17 20:38:42 INFO mlflow.tracking.fluent: Experiment with name '4-Model-Registry' does not exist. Creating a new experiment.


Client tracking uri: http://localhost:5001


<Experiment: artifact_location='mlflow-artifacts:/994473481243827653', creation_time=1729190322703, experiment_id='994473481243827653', last_update_time=1729190322703, lifecycle_stage='active', name='4-Model-Registry', tags={}>

## Registering a Model

- Docs on [mlflow.sklearn.log_model](https://www.mlflow.org/docs/latest/python_api/mlflow.sklearn.html?highlight=save_model#mlflow.sklearn.log_model)

### Log the `model_lr` model

In [11]:
with mlflow.start_run() as run: 
    
    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_lr,
        artifact_path="LinearRegression"
    )

2024/10/17 20:38:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/17 20:38:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run unequaled-cat-317 at: http://localhost:5001/#/experiments/994473481243827653/runs/8379b07022da4bcab663c6c7743a080c.
2024/10/17 20:38:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/994473481243827653.


### Log and Register the `model_rf` model (3 times)

- Use `registered_model_name` to register a model automatically.
- If a registered model with the name doesn’t exist, the method registers a new model and creates `Version 1`.
- If a registered model with the name exists, the method creates a new model version.

INSTRUCTION: 
- Run the cell below 3 time to register 3 versions of the "bike-sharing-RandomForestRegressor" model

In [15]:
from mlflow.models import infer_signature

with mlflow.start_run() as run: 

    # Show newly created run metadata info
    print("Experiment id: {}".format(run.info.experiment_id))
    print("Run id: {}".format(run.info.run_id))
    print("Run name: {}".format(run.info.run_name))
    print('MLFlow tracking uri:', mlflow.get_tracking_uri())
    print('MLFlow artifact uri:', mlflow.get_artifact_uri())
    run_id = run.info.run_id

    # Infer the model signature
    signature = infer_signature(y_test, preds_rf)

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model_rf,
        artifact_path="RandomForest",
        signature=signature,
        registered_model_name="RandomForest",
    )

Experiment id: 994473481243827653
Run id: afb821e5067c4ef199ee2221809a2288
Run name: powerful-snipe-471
MLFlow tracking uri: http://localhost:5001
MLFlow artifact uri: mlflow-artifacts:/994473481243827653/afb821e5067c4ef199ee2221809a2288/artifacts


Registered model 'RandomForest' already exists. Creating a new version of this model...
2024/10/17 20:39:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest, version 3
Created version '3' of model 'RandomForest'.
2024/10/17 20:39:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run powerful-snipe-471 at: http://localhost:5001/#/experiments/994473481243827653/runs/afb821e5067c4ef199ee2221809a2288.
2024/10/17 20:39:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/994473481243827653.


In [16]:
# Adding or Updating an MLflow Model Descriptions

client.update_model_version(
    name="RandomForest",
    version=3,
    description="This a model version 3 description added with update_model_version() method",
)

# Note: if you got an error, make sure you run the previous cell 3 times and save 3 versions of the model! 

<ModelVersion: aliases=[], creation_timestamp=1729190381215, current_stage='None', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1729190381960, name='RandomForest', run_id='afb821e5067c4ef199ee2221809a2288', run_link='', source='mlflow-artifacts:/994473481243827653/afb821e5067c4ef199ee2221809a2288/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='3'>

## Discover models and their stages

In [17]:
from pprint import pprint

# Listing and Searching MLflow Models

for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1729187700020,
    'description': '',
    'last_updated_timestamp': 1729189398033,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1729189398033, current_stage='None', description='', last_updated_timestamp=1729189398033, name='1-get-started-random-forest', run_id='1c0ee65e719445fbb7c53027741cf66d', run_link='', source='mlflow-artifacts:/0/1c0ee65e719445fbb7c53027741cf66d/artifacts/rf_model', status='READY', status_message='', tags={}, user_id='', version='2'>],
    'name': '1-get-started-random-forest',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1729190331423,
    'description': '',
    'last_updated_timestamp': 1729190381215,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1729190381215, current_stage='None', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1729190381960, name='RandomForest', run_id='afb821e5

In [18]:
# Search for a specific model name and list its version details 

for mv in client.search_model_versions("name='RandomForest'"):
    pprint(dict(mv), indent=4)

{   'aliases': [],
    'creation_timestamp': 1729190381215,
    'current_stage': 'None',
    'description': 'This a model version 3 description added with '
                   'update_model_version() method',
    'last_updated_timestamp': 1729190381960,
    'name': 'RandomForest',
    'run_id': 'afb821e5067c4ef199ee2221809a2288',
    'run_link': '',
    'source': 'mlflow-artifacts:/994473481243827653/afb821e5067c4ef199ee2221809a2288/artifacts/RandomForest',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '3'}
{   'aliases': [],
    'creation_timestamp': 1729190379945,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1729190379945,
    'name': 'RandomForest',
    'run_id': '1aee443c4ccd4c60bea297a83ec25525',
    'run_link': '',
    'source': 'mlflow-artifacts:/994473481243827653/1aee443c4ccd4c60bea297a83ec25525/artifacts/RandomForest',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    

## Transitioninig a model stage

In [19]:
# Over the course of the model’s lifecycle, a model evolves—from development to staging to production. 
# You can transition a registered model to one of the stages: Staging, Production or Archived.

client.transition_model_version_stage(
    name="RandomForest", version=3, stage="Production"
)

/var/folders/28/ly259mjn413_8g33c4n20tvc0000gn/T/ipykernel_11047/15454823.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1729190381215, current_stage='Production', description='This a model version 3 description added with update_model_version() method', last_updated_timestamp=1729190440209, name='RandomForest', run_id='afb821e5067c4ef199ee2221809a2288', run_link='', source='mlflow-artifacts:/994473481243827653/afb821e5067c4ef199ee2221809a2288/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='3'>

## Download and use models from the registry

In [20]:
model_version_uri = "models:/example-model@Champion"

In [21]:
# Load the model from the model registry and score
model_uri = f"models:/RandomForest/3"
loaded_model = mlflow.sklearn.load_model(model_uri)
loaded_model

RandomForestRegressor(n_estimators=50, random_state=0)

In [22]:
loaded_model.predict(X_test)

array([ 55.94,  49.42,   4.68,  31.98,  53.1 , 139.84, 129.62,  22.1 ,
       100.26,  62.8 ,  73.24,  62.8 ,  69.68, 100.6 ,  42.22,  70.56,
       103.14,  84.72, 112.4 , 185.3 , 152.9 ,   4.08,  55.96, 112.56,
        10.82,  57.22,  22.08,  89.2 ,  75.88,   7.52,  27.64, 154.14,
        69.16,  18.66,   6.74,  70.22,  94.92, 157.36,  64.8 ,  39.32,
        69.7 , 149.48, 174.44,  36.32,  36.76,   1.84,   8.74, 112.76,
        80.34,  50.28,  25.62,   4.14,  43.96,  52.2 , 117.96,   1.74,
        66.66,  72.1 ,  63.06,   3.22, 126.64,   4.96,  77.8 , 152.08,
         3.98,   5.34,  46.34,  56.4 ,  73.38,  79.86,  10.  ,  13.3 ,
       147.46,   3.3 ,  63.96, 159.26,  33.82,  31.52,   2.62,   3.68,
       112.02,  17.68,  61.92,  61.14,  67.56,  66.66,   2.76,  59.1 ,
         5.52,  78.38,  63.16,   1.8 ,  31.  ,  50.64,  28.1 , 145.54,
        82.56,  30.14,  17.22,  76.08,  73.9 ,  78.42,  98.26,  72.52,
        65.7 ,  45.48,  34.86,  19.38,  21.9 ,  53.44,   2.32,   2.44,
      

## Deregistering, Deleting and Archiving models 

In [23]:
# Archive models version 3 from Production into Archived

client = MlflowClient()
client.transition_model_version_stage(
    name="RandomForest", version=2, stage="Archived"
)

/var/folders/28/ly259mjn413_8g33c4n20tvc0000gn/T/ipykernel_11047/1646888081.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1729190379945, current_stage='Archived', description='', last_updated_timestamp=1729190486972, name='RandomForest', run_id='1aee443c4ccd4c60bea297a83ec25525', run_link='', source='mlflow-artifacts:/994473481243827653/1aee443c4ccd4c60bea297a83ec25525/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='2'>

In [24]:
# Note: Deleting registered models or model versions is irrevocable, so use it judiciously.

# Delete version 1 of the model
client.delete_model_version(
    name="RandomForest", version=1
)